In [1]:
# ======================================================================================================
# PROBLEM A3
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning.
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 97%.
# =======================================================================================================

import urllib.request
import zipfile
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


def solution_A3():
    inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

    pre_trained_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('mixed7')

    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')
    zip_ref.close()

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    train_dir = 'data/horse-or-human'
    validation_dir = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

    validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

    x = layers.Flatten()(last_layer.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        verbose=2)

    return model


if __name__ == '__main__':
    model = solution_A3()
    model.save("model_A3.h5")

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Epoch 1/10
52/52 - 86s - loss: 0.8963 - acc: 0.9396 - val_loss: 0.0021 - val_acc: 1.0000 - 86s/epoch - 2s/step
Epoch 2/10
52/52 - 80s - loss: 0.1246 - acc: 0.9679 - val_loss: 0.6135 - val_acc: 0.9375 - 80s/epoch - 2s/step
Epoch 3/10
52/52 - 79s - loss: 0.1428 - acc: 0.9747 - val_loss: 0.0759 - val_acc: 0.9883 - 79s/epoch - 2s/step
Epoch 4/10
52/52 - 90s - loss: 0.0872 - acc: 0.9864 - val_loss: 0.2415 - val_acc: 0.9727 - 90s/epoch - 2s/step
Epoch 5/10
52/52 - 82s - loss: 0.0607 - acc: 0.9903 - val_loss: 0.3214 - val_acc: 0.9609 - 82s/epoch - 2s/step
Epoch 6/10
52/52 - 80s - loss: 0.0630 - acc: 0.9873 - val_loss: 0.0899 - val_acc: 0.9844 - 80s/epoch - 2s/step
Epoch 7/10
52/52 - 80s - loss: 0.0360 - acc: 0.9942 - val_loss: 0.0162 - val_acc: 0.9961 - 80s/epoch - 2s/step
Epoch 8/10
52/52 - 80s - loss: 0.0458 - acc: 0.9873 - val_loss: 0.1981 - val_acc: 0.9727 - 80s/epoch - 2s/step
Epoch 9/10
52/52 - 80s - loss: 0.0063 - acc: 0.9981 - val_loss: 0.0870 - val_acc: 0.9922 - 80s/epoch - 2s/step
E